Getting Started with Distributed Data Parallel  
분산 데이터 병렬 시작

DistributedDataParallel (DDP) implements data parallelism at the module level which can run across multiple machines. Applications using DDP should spawn multiple processes and create a single DDP instance per process. DDP uses collective communications in the torch.distributed package to synchronize gradients and buffers. More specifically, DDP registers an autograd hook for each parameter given by model.parameters() and the hook will fire when the corresponding gradient is computed in the backward pass. Then DDP uses that signal to trigger gradient synchronization across processes. Please refer to DDP design note for more details.

The recommended way to use DDP is to spawn one process for each model replica, where a model replica can span multiple devices. DDP processes can be placed on the same machine or across machines, but GPU devices cannot be shared across processes. This tutorial starts from a basic DDP use case and then demonstrates more advanced use cases including checkpointing models and combining DDP with model parallel.

DDP (DistributedDataParallel)는 여러 시스템에서 실행 될 수있는 모듈 수준에서 데이터 병렬 처리를 구현합니다. DDP를 사용하는 응용 프로그램은 여러 프로세스를 생성하고 프로세스 당 단일 DDP 인스턴스를 만들어야합니다. DDP는 torch.distributed 패키지의 집단 통신을 사용하여 그라디언트 및 버퍼를 동기화합니다. 보다 구체적으로, DDP는 model.parameters()에 의해 주어진 각 매개 변수에 대해 autograd 훅을 등록하고 해당 그라디언트가 역방향 패스에서 계산 될 때 훅이 발생합니다. 그런 다음 DDP는 이 신호를 사용하여 프로세스 간 그래디언트 동기화를 트리거합니다. 자세한 내용은 DDP 디자인 노트를 참조하십시오.

DDP를 사용하는 데 권장되는 방법은 모델 복제본이 여러 장치에 걸쳐있을 수 있는 각 모델 복제본에 대해 하나의 프로세스를 생성하는 것 입니다. DDP 프로세스는 동일한 컴퓨터 또는 여러 컴퓨터에 배치 할 수 있지만 GPU 장치는 프로세스 간에 공유 할 수 없습니다. 이 학습서는 기본 DDP 유스 케이스에서 시작하여 체크 포인트 모델 및 DDP와 모델 병렬 결합을 포함하여보다 고급 유스 케이스를 보여줍니다.


Comparison between DataParallel and DistributedDataParallel

Before we dive in, let’s clarify why, despite the added complexity, you would consider using DistributedDataParallel over DataParallel:

    First, DataParallel is single-process, multi-thread, and only works on a single machine, while DistributedDataParallel is multi-process and works for both single- and multi- machine training. DataParallel is usually slower than DistributedDataParallel even on a single machine due to GIL contention across threads, per-iteration replicated model, and additional overhead introduced by scattering inputs and gathering outputs.

    Recall from the prior tutorial that if your model is too large to fit on a single GPU, you must use model parallel to split it across multiple GPUs. DistributedDataParallel works with model parallel; DataParallel does not at this time. When DDP is combined with model parallel, each DDP process would use model parallel, and all processes collectively would use data parallel.

    If your model needs to span multiple machines or if your use case does not fit into data parallelism paradigm, please see the RPC API for more generic distributed training support.

DataParallel과 DistributedDataParallel의 비교

자세히 알아보기 전에 복잡성이 추가 되었음에도 불구하고 DataParallel 대신 DistributedDataParallel을 사용하는 이유를 명확히 하겠습니다.

첫째, DataParallel은 단일 프로세스, 다중 스레드이며 단일 시스템에서만 작동하는 반면, DistributedDataParallel은 다중 프로세스이며 단일 및 다중 시스템 훈련에 모두 작동합니다. 스레드 간에 GIL 경합, 반복 복제 모델 및 입력 분산 및 출력 수집으로 인한 추가 오버 헤드로 인해 단일 시스템에서도 DataParallel이 DistributedDataParallel보다 일반적으로 느립니다.

이전 자습서에서 모델이 너무 커서 단일 GPU에 맞지 않으면 모델을 병렬로 사용하여 여러 GPU로 분할해야 합니다. DistributedDataParallel은 모델 병렬로 작동합니다. 현재 DataParallel은 없습니다. DDP가 모델 병렬과 결합되면 각 DDP 프로세스는 모델 병렬을 사용하고 모든 프로세스는 집합 적으로 데이터 병렬을 사용합니다.

모델이 여러 시스템에 걸쳐 있거나 사용 사례가 데이터 병렬 패러다임에 맞지 않는 경우 보다 일반적인 분산 교육 지원에 대해서는 RPC API를 참조하십시오.

Basic Use Case

To create DDP modules, first set up process groups properly. More details can be found in Writing Distributed Applications with PyTorch.

DDP 모듈을 작성하려면 먼저 프로세스 그룹을 올바르게 설정하십시오. 자세한 내용은 PyTorch로 분산 응용 프로그램 작성에서 찾을 수 있습니다.

In [9]:
import os
import tempfile
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp

from torch.nn.parallel import DistributedDataParallel as DDP

def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    
    # initialize the process group
    dist.init_process_group('gloo', rank=rank, world_size=world_size)
    
def cleanup():
    dist.destroy_process_group()

Now, let’s create a toy module, wrap it with DDP, and feed it with some dummy input data. Please note, as DDP broadcasts model states from rank 0 process to all other processes in the DDP constructor, you don’t need to worry about different DDP processes start from different model parameter initial values.

이제 toy 모듈을 만들어 DDP로 감싸서 더미 입력 데이터를 공급해 보겠습니다. DDP는 순위 0 프로세스에서 DDP 생성자에 있는 다른 모든 프로세스로 모델 상태를 브로드 캐스트하므로 다른 모델 매개 변수 초기 값에서 시작하는 다른 DDP 프로세스에 대해 걱정할 필요가 없습니다.

In [10]:
class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel, self).__init__()
        self.net1 = nn.Linear(10, 10)
        self.relu = nn.ReLU()
        self.net2 = nn.Linear(10, 5)
        
    def forward(self, x):
        return self.net2(self.relu(self.net1(x)))
    
def demo_basic(rank, world_size):
    print(f'Running basic DDP example on rank {rank}.')
    setup(rank, world_size)
    
    # create model and move it to GPU with id rank
    model = ToyModel().to(rank)
    ddp_model = DDP(model, device_ids=[rank])
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)
    
    optimizer.zero_grad()
    outputs = ddp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(rank)
    loss_fn(outputs, labels).backward()
    optimizer.step()
    
    clearup()
    
def run_demo(demo_fn, world_size):
    mp.spawn(demo_fn, args=(world_size,), nprocs=world_size, join=True)

As you can see, DDP wraps lower-level distributed communication details and provides a clean API as if it is a local model. Gradient synchronization communications take place during the backward pass and overlap with the backward computation. When the backward() returns, param.grad already contains the synchronized gradient tensor. For basic use cases, DDP only requires a few more LoCs to set up the process group. When applying DDP to more advanced use cases, some caveats require caution.

보다시피 DDP는 하위 수준의 분산 통신 세부 정보를 래핑하고 마치 로컬 모델인 것 처럼 깨끗한 API를 제공합니다. 그라디언트 동기화 통신은 역방향 패스 동안 발생하며 역방향 계산과 겹칩니다. backward()이 반환되면 param.grad에 이미 동기화 된 그래디언트 텐서가 포함되어 있습니다. 기본 사용 사례의 경우 DDP는 프로세스 그룹을 설정하기 위해 몇 개의 LoC 만 필요합니다. 고급 사용 사례에 DDP를 적용 할 때 주의해야 할 사항이 있습니다.

Skewed Processing Speeds  

In DDP, the constructor, the forward pass, and the backward pass are distributed synchronization points. Different processes are expected to launch the same number of synchronizations and reach these synchronization points in the same order and enter each synchronization point at roughly the same time. Otherwise, fast processes might arrive early and timeout on waiting for stragglers. Hence, users are responsible for balancing workloads distributions across processes. Sometimes, skewed processing speeds are inevitable due to, e.g., network delays, resource contentions, unpredictable workload spikes. To avoid timeouts in these situations, make sure that you pass a sufficiently large timeout value when calling init_process_group.

DDP 에서 생성자, 전달 패스 및 역방향 패스는 분산 동기화 지점입니다. 서로 다른 프로세스는 동일한 수의 동기화를 시작하고 동일한 순서로 이러한 동기화 지점에 도달하고 대략 동시에 각 동기화 지점에 들어갈 것으로 예상됩니다. 그렇지 않으면 빠른 프로세스가 일찍 도착하여 straggler 대기 시간이 초과 될 수 있습니다. 따라서 사용자는 프로세스 간 워크로드 분산의 균형을 유지 해야 합니다. 때로는 네트워크 지연, 리소스 경합, 예측할 수 없는 워크로드 급증으로 인해 처리 속도가 왜곡 될 수 있습니다. 이러한 상황에서 시간 초과를 피하려면 init_process_group을 호출 할 때 시간 초과 값을 충분히 크게 전달하십시오.

Save and Load Checkpoints  

It’s common to use torch.save and torch.load to checkpoint modules during training and recover from checkpoints. See SAVING AND LOADING MODELS for more details. When using DDP, one optimization is to save the model in only one process and then load it to all processes, reducing write overhead. This is correct because all processes start from the same parameters and gradients are synchronized in backward passes, and hence optimizers should keep setting parameters to the same values. If you use this optimization, make sure all processes do not start loading before the saving is finished. Besides, when loading the module, you need to provide an appropriate map_location argument to prevent a process to step into others’ devices. If map_location is missing, torch.load will first load the module to CPU and then copy each parameter to where it was saved, which would result in all processes on the same machine using the same set of devices. For more advanced failure recovery and elasticity support, please refer to TorchElastic.

훈련 중에 checkpoints 모듈을 검사하고 checkpoints 에서 복구하는 데 torch.save 및 torch.load를 사용하는 것이 일반적입니다. 자세한 내용은 저장 및 로딩 모델을 참조하십시오. DDP를 사용할 때 하나의 최적화는 모델을 하나의 프로세스에만 저장 한 다음 모든 프로세스에 로드하여 쓰기 오버 헤드를 줄이는 것입니다. 모든 프로세스가 동일한 매개 변수에서 시작하고 그라디언트가 역방향 패스로 동기화되므로 최적화 프로그램이 매개 변수를 동일한 값으로 설정해야합니다. 이 최적화를 사용하는 경우 저장이 완료되기 전에 모든 프로세스가 로드를 시작하지 않는지 확인하십시오. 또한 모듈을 로드 할 때 프로세스가 다른 기기로 들어 가지 않도록 적절한 map_location 인수를 제공 해야 합니다. map_location이 없으면 torch.load는 먼저 모듈을 CPU에 로드 한 다음 각 매개 변수를 저장된 위치에 복사하여 동일한 장치에서 동일한 장치 세트를 사용하는 모든 프로세스를 생성합니다. 고급 오류 복구 및 탄력성 지원에 대해서는 TorchElastic을 참조하십시오.

In [11]:
def demo_checkpoint(rank, world_size):
    print(f'Running DDP checkpoint example on rank {rank}')
    setup(rank, world_size)
    
    model = ToyModel().to(rank)
    ddp_model = DDP(model, device_ids=[rank])
    
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)
    
    CHECKPOINT_PATH = tempfile.gettempdir() + '/model.checkpoint'
    if rank == 0:
        # 모든 프로세스는 모두 동일한 매개 변수로 시작해야합니다.
        # 랜덤 파라미터와 그라디언트는 역방향 패스에서 동기화됩니다.
        # 따라서 하나의 프로세스로 저장하면 충분합니다.
        torch.save(ddp_model.state_dict(), CHECKPOINT_PATH)
        
    # process 0 이 모델을 저장 한 후 process 1 이 모델을 로드하는지 확인하려면 
    # barrier()를 사용하십시오.
    dist.barrier()
    
    # configure map_location properly
    map_location = {'cuda:%d' % 0 : 'cuda:%d' % rank}
    ddp_model.load_state_dict(
        torch.load(CHECKPOINT_PATH, map_location=map_location))
    
    optimizer.zero_grad()
    outputs = ddp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(rank)
    loss_fn = nn.MSELoss()
    loss_fn(outputs, labels).backward()
    optimizer.step()
    
    # Use a barrier() to make sure that 
    # all processes have finished reading the checkpoint.
    dist.barrier()
    
    if rank == 0:
        os.remove(CHECKPOINT_PATH)
        
    clean_up()

Combine DDP with Model Parallelism  

DDP also works with multi-GPU models. DDP wrapping multi-GPU models is especially helpful when training large models with a huge amount of data.


DDP는 다중 GPU 모델에서도 작동합니다. 다중 GPU 모델을 래핑하는 DDP는 많은 양의 데이터로 큰 모델을 훈련 할 때 특히 유용합니다.

In [13]:
class ToyMpModel(nn.Module):
    def __init__(self, dev0, dev1):
        super(ToyMpModel, self).__init__()
        self.dev0 = dev0
        self.dev1 = dev1
        self.net1 = torch.nn.Linear(10, 10).to(dev0)
        self.relu = torch.nn.ReLU()
        self.net2 = torch.nn.Linear(10, 5).to(dev1)
        
    def forward(self, x):
        x = x.to(self.dev0)
        x = self.relu(self.net1(x))
        x = x.to(self.dev1)
        return self.net2(x)

When passing a multi-GPU model to DDP, device_ids and output_device must NOT be set. Input and output data will be placed in proper devices by either the application or the model forward() method.

다중 GPU 모델을 DDP에 전달할 때는 device_ids 및 output_device를 설정하지 않아야 합니다. 입력 및 출력 데이터는 응용 프로그램 또는 모델 forward() 메소드에 의해 적절한 장치에 배치됩니다.

In [14]:
def demo_model_parallel(rank, world_size):
    print(f'Running DDP with model parallel example on rank {rank}')
    setup(rank, world_size)
    
    # setup mp_model and devices for this process
    dev0 = rank * 2
    dev1 = rank * 2 + 1
    mp_model = ToyMpModel(dev0, dev1)
    ddp_mp_model = DDP(mp_model)
    
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_mp_model.parameters(), lr=0.001)
    
    optimizer.zero_grad()
    # outputs will be on dev1
    outputs = ddp_mp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(dev1)
    loss_fn(outputs, labels).backward()
    optimizer.step()
    
    cleanup()
    
if __name__ == '__main__':
    n_gpus = torch.cuda.device_count()
    run_demo(demo_basic, 2)

Exception: process 1 terminated with exit code 1